In [1]:
import os
import math
import datetime
import numpy as np
import pandas as pd


import time
import os.path

## step 1 obtain OSM network

In [2]:
"Get the OSM Network"
network_type = 'auto'
import osm2gmns as og
files= os.listdir(os.getcwd()) 
for file in files: 
    if file[-7:] == 'utd.osm':
        net = og.getNetFromOSMFile(file,network_type=(network_type), default_lanes=True, default_speed=True)
        break

og.consolidateComplexIntersections(net)
og.outputNetToCSV(net, output_folder=os.getcwd())

osm2gmns, version 0.4.2


## step 2 Convert utd Data into node and link

In [51]:
"Convert utd Data into GMNS Format"
files= os.listdir(os.getcwd()) 
for file in files: 
    if file[:5] == 'links':
        utd = pd.read_csv(file)
    if file[:9] == 'detectors':
        detectors = pd.read_csv(file)
        break
utd.rename(columns = {'Unnamed: 0':'utd','long':'utd_long','lat':'utd_lat','citycode':'utd_citycode'}, inplace = True)
detectors.rename(columns = {'Unnamed: 0':'detectors','long':'detectors_long',\
                'lat':'detectors_lat','citycode':'detectors_citycode','length':'detectors_length'}, inplace = True)
detectors['linkid'] = detectors['linkid'].astype(int)
detectors['lanes'] = detectors['lanes'].astype(int)


detectors = detectors[-detectors['linkid'].duplicated()]
# detectors = detectors.drop(detectors[detectors['detid']=='N29121Z'].index) #dupicate and error
utd_detectors = utd.merge(detectors, on='linkid', how='left')  #miss one detector 89950

In [4]:
num_link = 0
'''build node.csv'''
print('converting utd data into gmns format...')


node_utd = pd.DataFrame()
node_utd['name'] = None
node_utd['x_coord'] = None
node_utd['y_coord'] = None
node_utd['z_coord'] = None
node_utd['node_type'] = None
node_utd['ctrl_type'] = None
node_utd['zone_id'] = None
node_utd['parent_node_id'] = None
node_utd['geometry'] = None

for i in range(len(utd)-1):
    if utd.loc[i+1,'order'] > utd.loc[i,'order']:
        node_utd = node_utd.append({'name': utd.loc[i,'utd'],\
                                    'x_coord': utd.loc[i,'utd_long'], \
                                    'y_coord': utd.loc[i,'utd_lat'],\
                                    'z_coord': None,\
                                    'node_type': utd.loc[i,'linkid'],\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + utd.loc[i,'utd_long'].astype(str) + " " + utd.loc[i,'utd_lat'].astype(str) +")"}, ignore_index=True)
        num_link += 1
    else:
        
        node_utd = node_utd.append({'name': utd.loc[i,'utd'],\
                                    'x_coord': utd.loc[i,'utd_long'], \
                                    'y_coord': utd.loc[i,'utd_lat'],\
                                    'z_coord': None,\
                                    'node_type': utd.loc[i,'linkid'],\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + utd.loc[i,'utd_long'].astype(str) + " " + utd.loc[i,'utd_lat'].astype(str) +")"}, ignore_index=True)

node_utd = node_utd.append({'name': utd.loc[i+1,'utd'],\
                                    'x_coord': utd.loc[i+1,'utd_long'], \
                                    'y_coord': utd.loc[i+1,'utd_lat'],\
                                    'z_coord': None,\
                                    'node_type': utd.loc[i+1,'linkid'],\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + utd.loc[i+1,'utd_long'].astype(str) + " " + utd.loc[i+1,'utd_lat'].astype(str) +")"}, ignore_index=True)


node_utd.index.name = 'node_id'
node_utd.index += 100000001 #index from 0



node_utd.to_csv('node_utd.csv')
print('node_utd.csv generated!')

converting utd data into gmns format...
node_utd.csv generated!


In [5]:
node_utd

,name,x_coord,y_coord,z_coord,node_type,ctrl_type,zone_id,parent_node_id,geometry
node_id,,,,,,,,,
100000001,89398,-2.264742,53.485628,None,0,None,None,None,POINT (-2.264742 53.48562800000001)
100000002,89399,-2.264694,53.486187,None,0,None,None,None,POINT (-2.264694 53.486187)
100000003,89400,-2.264653,53.486820,None,0,None,None,None,POINT (-2.264653 53.48682)
100000004,89401,-2.264374,53.487190,None,0,None,None,None,POINT (-2.264374 53.48719000000001)
100000005,89402,-2.263860,53.487496,None,0,None,None,None,POINT (-2.26386 53.48749599999999)
...,...,...,...,...,...,...,...,...,...
100000938,90335,-2.252004,53.481483,None,180,None,None,None,POINT (-2.2520041 53.48148270000001)
100000939,90336,-2.252378,53.481538,None,180,None,None,None,POINT (-2.252378 53.481538)
100000940,90337,-2.252817,53.481670,None,180,None,None,None,POINT (-2.2528169 53.4816695)


In [6]:
'''build link_utd.csv'''
link_utd = pd.DataFrame()
link_utd['name'] = None
link_utd['corridor_id'] = None
link_utd['corridor_link_order'] = None
link_utd['from_node_id'] = None
link_utd['to_node_id'] = None
link_utd['directed'] = None
link_utd['geometry_id'] = None
link_utd['geometry'] = None
link_utd['dir_flag'] = None
link_utd['parent_link_id'] = None
link_utd['length'] = None
link_utd['grade'] = None
link_utd['facility_type'] = None
link_utd['capacity'] = None
link_utd['free_speed'] = None
link_utd['lanes'] = None
link_utd['detid'] = None

for i in range(0,len(utd_detectors)-1):
    if utd_detectors.loc[i+1,'order'] > utd_detectors.loc[i,'order']:
        link_utd = link_utd.append({'name': utd_detectors.loc[i,'utd'],\
                                    'corridor_id': utd_detectors.loc[i,'road'],\
                                    'corridor_link_order' : utd_detectors.loc[i,'order'],\
                                    'from_node_id': node_utd[(node_utd['x_coord']==utd_detectors.loc[i,'utd_long']) & \
                                                    (node_utd['y_coord']==utd_detectors.loc[i,'utd_lat'])].index.values[0], \
                                    'to_node_id': node_utd[(node_utd['x_coord']==utd_detectors.loc[i+1,'utd_long']) & \
                                                    (node_utd['y_coord']==utd_detectors.loc[i+1,'utd_lat'])].index.values[0],\
                                    'directed': None,\
                                    'geometry_id': None,\
                                    'geometry': "LINESTRING (" + utd_detectors.loc[i,'utd_long'].astype(str) + " " + \
                                                utd_detectors.loc[i,'utd_lat'].astype(str) + "," + \
                                                utd_detectors.loc[i+1,'utd_long'].astype(str) +" "+ \
                                                utd_detectors.loc[i+1,'utd_lat'].astype(str) + ")",\
                                    'dir_flag': None,\
                                    'parent_link_id': None,\
                                    'length': None,\
                                    'grade': utd_detectors.loc[i,'fclass'],\
                                    'facility_type': None ,\
                                    'capacity':None,\
                                    'free_speed':utd_detectors.loc[i,'limit'],\
                                    'lanes': utd_detectors.loc[i,'lanes'],\
                                    'detid': utd_detectors.loc[i,'detid']}, ignore_index=True)
link_utd.index.name = 'link_id'
link_utd.index += 100000001


link_utd.to_csv('link_utd.csv')
print('link_utd.csv generated!')


link_utd.csv generated!


In [7]:
link_utd

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,directed,geometry_id,geometry,dir_flag,parent_link_id,length,grade,facility_type,capacity,free_speed,lanes,detid
link_id,,,,,,,,,,,,,,,,,
100000001,89398,Silk Street,1,100000001,100000002,None,None,"LINESTRING (-2.264742 53.48562800000001,-2.264...",None,None,None,primary,None,None,48.0,1.0,N31111S
100000002,89399,Silk Street,2,100000002,100000003,None,None,"LINESTRING (-2.264694 53.486187,-2.264653 53.4...",None,None,None,primary,None,None,48.0,1.0,N31111S
100000003,89400,Silk Street,3,100000003,100000004,None,None,"LINESTRING (-2.264653 53.48682,-2.264374 53.48...",None,None,None,primary,None,None,48.0,1.0,N31111S
100000004,89401,Silk Street,4,100000004,100000005,None,None,"LINESTRING (-2.264374 53.48719000000001,-2.263...",None,None,None,primary,None,None,48.0,1.0,N31111S
100000005,89402,Silk Street,5,100000005,100000006,None,None,"LINESTRING (-2.26386 53.48749599999999,-2.2628...",None,None,None,primary,None,None,48.0,1.0,N31111S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100000757,90333,Old Mill Street,2,100000936,100000937,None,None,"LINESTRING (-2.223232 53.48135500000001,-2.224...",None,None,None,tertiary,None,None,0.0,1.0,N06271H
100000758,90335,Leftbank,1,100000938,100000939,None,None,"LINESTRING (-2.2520041 53.48148270000001,-2.25...",None,None,None,other,None,None,48.0,1.0,N06521G
100000759,90336,Leftbank,2,100000939,100000940,None,None,"LINESTRING (-2.252378 53.481538,-2.2528169 53....",None,None,None,other,None,None,48.0,1.0,N06521G


## step 3 mapping osm network to utd network

In [8]:
link_road = pd.read_csv('link.csv', low_memory=False)

In [9]:
def LLs2Dist(lon1, lat1, lon2, lat2): #WGS84 transfer coordinate system to distance(meter) #xy #credit to xtHuang0927
    R = 6371
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0

    a = math.sin(dLat / 2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180.0) * math.cos(lat2 * math.pi / 180.0) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c * 1000
    return distance

In [10]:
for i in range(len(link_road)):
    lon_list = []
    lat_list = [] 
    link_geometry_list = link_road.loc[i,'geometry'][12:-1].split(", ")
    for link_geometry in link_geometry_list:
        lon_list.append(float(link_geometry.split(" ")[0]))
        lat_list.append(float(link_geometry.split(" ")[1]))
    center_lon = np.mean(lon_list)
    center_lat = np.mean(lat_list)


    distance_list = []
    for j in link_utd.index:
        lon_utd_list = []
        lat_utd_list = []
        link_utd_geometry_list = link_utd.loc[j,'geometry'][12:-1].split(",")
        for link_utd_geometry in link_utd_geometry_list:
            lon_utd_list.append(float(link_utd_geometry.split(" ")[0]))
            lat_utd_list.append(float(link_utd_geometry.split(" ")[1]))
        center_utd_lon = np.mean(lon_utd_list)
        center_utd_lat = np.mean(lat_utd_list)
        
        distance_list.append(LLs2Dist(center_lon, center_lat, center_utd_lon, center_utd_lat))
    nearest_index = distance_list.index(min(distance_list))

    link_road.loc[i,'detid'] = link_utd.iloc[nearest_index]['detid']
    link_road.loc[i,'utd_link_id'] = int(link_utd.iloc[[nearest_index]].index.values[0])
    link_road.loc[i,'matched_from_node_id'] = int(link_utd.iloc[nearest_index]['from_node_id'])
    link_road.loc[i,'matched_to_node_id'] = int(link_utd.iloc[nearest_index]['to_node_id'])





In [11]:
link_road

,name,link_id,osm_way_id,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses,from_biway,detid,utd_link_id,matched_from_node_id,matched_to_node_id
0,Cobourg Street,0,796758,110,109,1,84.376464,1,30,NaN,residential,6,"LINESTRING (-2.2331957 53.4759624, -2.2332846 ...",auto,0,N06271L,100000402.0,100000507.0,100000508.0
1,South Pump Street,1,797061,1891,117,1,36.928887,1,30,NaN,residential,6,"LINESTRING (-2.2320231 53.4765284, -2.2318448 ...",auto,1,N06271L,100000402.0,100000507.0,100000508.0
2,South Pump Street,2,797061,117,1891,1,36.928887,1,30,NaN,residential,6,"LINESTRING (-2.2318448 53.4762137, -2.2320231 ...",auto,1,N06271L,100000402.0,100000507.0,100000508.0
3,Echo Street,3,797062,118,1899,1,3.911961,1,29,NaN,unclassified,11,"LINESTRING (-2.2316462 53.4757191, -2.2316626 ...",auto,0,N06211F,100000385.0,100000486.0,100000487.0
4,Granby Row,4,797065,98,1450,1,61.506079,1,20,NaN,residential,6,"LINESTRING (-2.2377205 53.4748575, -2.2370128 ...",auto,1,N06211F,100000381.0,100000482.0,100000483.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5404,Thor Grove,5419,953192214,2439,2440,1,9.640846,1,20,NaN,residential,6,"LINESTRING (-2.2687887 53.4746256, -2.268925 5...",auto,1,N29131F,100000687.0,100000854.0,100000855.0
5405,Asgard Grove,5420,953195491,2442,2441,1,12.007100,1,20,NaN,residential,6,"LINESTRING (-2.268867 53.4756163, -2.2690335 5...",auto,1,N29131F,100000690.0,100000857.0,100000858.0
5406,Asgard Grove,5421,953195491,2441,2442,1,12.007100,1,20,NaN,residential,6,"LINESTRING (-2.2690335 53.4756592, -2.268867 5...",auto,1,N29131F,100000690.0,100000857.0,100000858.0
5407,NaN,5422,953707228,2444,2443,1,88.415392,1,20,NaN,residential,6,"LINESTRING (-2.2666748 53.4757732, -2.2663271 ...",auto,1,N29121N,100000701.0,100000869.0,100000870.0


## step 4 generating measurement_osm and measurement_utd

In [19]:
## reading by detid
files= os.listdir(os.getcwd()) 
for file in files: 
    if file[:4] == 'data':
        reading = pd.read_csv(file)
        break

# reading = reading[pd.to_datetime(reading['measurement_tstamp'], format='%Y-%m-%d %H:%M:%S')<datetime.datetime.strptime('2015-04-01 02:00:00', '%Y-%m-%d %H:%M:%S')]
reading = reading.loc[0:3000]


reading_dict = {}
gp = reading.groupby('detid')
for key, form in gp:
    reading_dict[key] = {
        'day':form['day'].tolist(),
        'interval':form['interval'].tolist(),
        'flow':form['flow'].tolist(),
        'occ':form['occ'].tolist(),
        'speed':form['speed'].tolist()
        }

In [20]:
'''build measurement_osm.csv''' 


measurement_osm = pd.DataFrame()
measurement_osm['link_id'] = None
measurement_osm['osm_way_id'] = None
measurement_osm['from_node_id'] = None
measurement_osm['to_node_id'] = None
measurement_osm['utd_link_id'] = None
measurement_osm['matched_from_node_id'] = None
measurement_osm['matched_to_node_id'] = None
measurement_osm['lanes'] = None
measurement_osm['length'] = None
measurement_osm['time_period'] = None
measurement_osm['date'] = None
measurement_osm['geometry'] = None
measurement_osm['volume'] = None
measurement_osm['speed'] = None




for i in link_road.index:
    try:
        reading_dict_selected = reading_dict[link_road['detid'][i]]
        for j in range(0,len(reading_dict_selected['day']),3):
            measurement_osm = measurement_osm.append({'link_id': link_road['link_id'][i],\
                                            'osm_way_id':link_road['osm_way_id'][i],\
                                            'from_node_id': link_road['from_node_id'][i],\
                                            'to_node_id': link_road['to_node_id'][i],\
                                            'utd_link_id':int(link_road['utd_link_id'][i]),\
                                            'matched_from_node_id': int(link_road['matched_from_node_id'][i]),\
                                            'matched_to_node_id': int(link_road['matched_to_node_id'][i]),\
                                            'lanes': link_road['lanes'][i], \
                                            'length': link_road['length'][i], \
                                            'time_period': str(divmod(reading_dict_selected['interval'][j], 3600)[0]).zfill(2)+\
                                                str(int(divmod(reading_dict_selected['interval'][j], 3600)[1]/60)).zfill(2)+'_'+\
                                                str(divmod(reading_dict_selected['interval'][j+3], 3600)[0]).zfill(2)+\
                                                str(int(divmod(reading_dict_selected['interval'][j+3], 3600)[1]/60)).zfill(2),\
                                            'date':reading_dict_selected['day'][j],\
                                            'geometry': link_road['geometry'][i],\
                                            'volume': np.sum(reading_dict_selected['flow'][j:j+3]),\
                                            'speed': round(np.mean(reading_dict_selected['speed'][j:j+3]))}, ignore_index=True)
    except:
        measurement_osm = measurement_osm.append({'link_id': link_road['link_id'][i],\
                                            'osm_way_id':link_road['osm_way_id'][i],\
                                            'from_node_id': link_road['from_node_id'][i],\
                                            'to_node_id': link_road['to_node_id'][i],\
                                            'utd_link_id':int(link_road['utd_link_id'][i]),\
                                            'matched_from_node_id': int(link_road['matched_from_node_id'][i]),\
                                            'matched_to_node_id': int(link_road['matched_to_node_id'][i]),\
                                            'lanes': link_road['lanes'][i], \
                                            'length': link_road['length'][i], \
                                            'time_period': None,\
                                            'date': None,\
                                            'geometry': link_road['geometry'][i],\
                                            'volume': None,\
                                            'speed': None}, ignore_index=True)

measurement_osm.to_csv('measurement_osm.csv',index = False)
print('measurement_osm.csv generated!')

measurement_osm.csv generated!


In [26]:
measurement_osm

,link_id,osm_way_id,from_node_id,to_node_id,utd_link_id,matched_from_node_id,matched_to_node_id,lanes,length,time_period,date,geometry,volume,speed
0,0,796758,110,109,100000402,100000507,100000508,1,84.376464,2310_2325,2017-09-29,"LINESTRING (-2.2331957 53.4759624, -2.2332846 ...",1110.0,40
1,0,796758,110,109,100000402,100000507,100000508,1,84.376464,2325_2340,2017-09-29,"LINESTRING (-2.2331957 53.4759624, -2.2332846 ...",1068.0,40
2,0,796758,110,109,100000402,100000507,100000508,1,84.376464,2340_2355,2017-09-29,"LINESTRING (-2.2331957 53.4759624, -2.2332846 ...",648.0,43
3,0,796758,110,109,100000402,100000507,100000508,1,84.376464,2355_0010,2017-09-29,"LINESTRING (-2.2331957 53.4759624, -2.2332846 ...",510.0,42
4,0,796758,110,109,100000402,100000507,100000508,1,84.376464,0010_0025,2017-09-30,"LINESTRING (-2.2331957 53.4759624, -2.2332846 ...",522.0,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32434,5423,953707228,2443,2444,100000701,100000869,100000870,1,88.415392,2325_2340,2017-09-29,"LINESTRING (-2.2660935 53.4752366, -2.2661771 ...",738.0,26
32435,5423,953707228,2443,2444,100000701,100000869,100000870,1,88.415392,2340_2355,2017-09-29,"LINESTRING (-2.2660935 53.4752366, -2.2661771 ...",834.0,27
32436,5423,953707228,2443,2444,100000701,100000869,100000870,1,88.415392,2355_0010,2017-09-29,"LINESTRING (-2.2660935 53.4752366, -2.2661771 ...",690.0,31
32437,5423,953707228,2443,2444,100000701,100000869,100000870,1,88.415392,0010_0025,2017-09-30,"LINESTRING (-2.2660935 53.4752366, -2.2661771 ...",750.0,35


In [24]:
'''build measurement_utd.csv''' 

measurement_utd = pd.DataFrame()
measurement_utd['utd_link_id'] = None
measurement_utd['corridor_id'] = None
measurement_utd['corridor_link_order'] = None
measurement_utd['from_node_id'] = None
measurement_utd['to_node_id'] = None
measurement_utd['time_period'] = None
measurement_utd['date'] = None
measurement_utd['geometry'] = None
measurement_utd['volume'] = None
measurement_utd['travel_time'] = None
measurement_utd['speed'] = None
measurement_utd['reference_speed'] = None
measurement_utd['density'] = None
measurement_utd['queue'] = None
measurement_utd['notes'] = None



for i in link_utd.index:
    try:
        reading_dict_selected = reading_dict[link_utd['detid'][i]]
        for j in range(0,len(reading_dict_selected['day']),3):
            measurement_utd = measurement_utd.append({'utd_link_id': i,\
                                            'corridor_id': link_utd['corridor_id'][i],\
                                            'corridor_link_order' : link_utd['corridor_link_order'][i],\
                                            'from_node_id': link_utd.loc[i,'from_node_id'], \
                                            'to_node_id': link_utd.loc[i,'to_node_id'], \
                                            'time_period': str(divmod(reading_dict_selected['interval'][j], 3600)[0]).zfill(2)+\
                                                str(int(divmod(reading_dict_selected['interval'][j], 3600)[1]/60)).zfill(2)+'_'+\
                                                str(divmod(reading_dict_selected['interval'][j+3], 3600)[0]).zfill(2)+\
                                                str(int(divmod(reading_dict_selected['interval'][j+3], 3600)[1]/60)).zfill(2),\
                                            'date': reading_dict_selected['day'][j],\
                                            'geometry': link_utd['geometry'][i],\
                                            'volume': np.sum(reading_dict_selected['flow'][j:j+3]),\
                                            'travel_time': None,\
                                            'speed': round(np.mean(reading_dict_selected['speed'][j:j+3])),\
                                            'reference_speed': None,\
                                            'density': None,\
                                            'queue': None,\
                                            'notes': None }, ignore_index=True)

    except:
        measurement_utd = measurement_utd.append({'utd_link_id': i,\
                                        'corridor_id': link_utd['corridor_id'][i],\
                                        'corridor_link_order' : link_utd['corridor_link_order'][i],\
                                        'from_node_id': link_utd.loc[i,'from_node_id'], \
                                        'to_node_id': link_utd.loc[i,'to_node_id'], \
                                        'time_period': None,\
                                        'date': None,\
                                        'geometry': link_utd['geometry'][i],\
                                        'volume': None,\
                                        'travel_time': None,\
                                        'speed': None,\
                                        'reference_speed': None,\
                                        'density': None,\
                                        'queue': None,\
                                        'notes': None }, ignore_index=True)

measurement_utd.to_csv('measurement_utd.csv',index = False)
print('measurement_utd.csv generated!')

measurement_utd.csv generated!


In [54]:
link_utd

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,directed,geometry_id,geometry,dir_flag,parent_link_id,length,grade,facility_type,capacity,free_speed,lanes,detid
link_id,,,,,,,,,,,,,,,,,
100000001,89398,Silk Street,1,100000001,100000002,None,None,"LINESTRING (-2.264742 53.48562800000001,-2.264...",None,None,None,primary,None,None,48.0,1.0,N31111S
100000002,89399,Silk Street,2,100000002,100000003,None,None,"LINESTRING (-2.264694 53.486187,-2.264653 53.4...",None,None,None,primary,None,None,48.0,1.0,N31111S
100000003,89400,Silk Street,3,100000003,100000004,None,None,"LINESTRING (-2.264653 53.48682,-2.264374 53.48...",None,None,None,primary,None,None,48.0,1.0,N31111S
100000004,89401,Silk Street,4,100000004,100000005,None,None,"LINESTRING (-2.264374 53.48719000000001,-2.263...",None,None,None,primary,None,None,48.0,1.0,N31111S
100000005,89402,Silk Street,5,100000005,100000006,None,None,"LINESTRING (-2.26386 53.48749599999999,-2.2628...",None,None,None,primary,None,None,48.0,1.0,N31111S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100000757,90333,Old Mill Street,2,100000936,100000937,None,None,"LINESTRING (-2.223232 53.48135500000001,-2.224...",None,None,None,tertiary,None,None,0.0,1.0,N06271H
100000758,90335,Leftbank,1,100000938,100000939,None,None,"LINESTRING (-2.2520041 53.48148270000001,-2.25...",None,None,None,other,None,None,48.0,1.0,N06521G
100000759,90336,Leftbank,2,100000939,100000940,None,None,"LINESTRING (-2.252378 53.481538,-2.2528169 53....",None,None,None,other,None,None,48.0,1.0,N06521G


In [25]:
measurement_utd

,utd_link_id,corridor_id,corridor_link_order,from_node_id,to_node_id,time_period,date,geometry,volume,travel_time,speed,reference_speed,density,queue,notes
0,100000001,Silk Street,1,100000001,100000002,2310_2325,2017-09-29,"LINESTRING (-2.264742 53.48562800000001,-2.264...",240,None,40,None,None,None,None
1,100000001,Silk Street,1,100000001,100000002,2325_2340,2017-09-29,"LINESTRING (-2.264742 53.48562800000001,-2.264...",228,None,36,None,None,None,None
2,100000001,Silk Street,1,100000001,100000002,2340_2355,2017-09-29,"LINESTRING (-2.264742 53.48562800000001,-2.264...",204,None,45,None,None,None,None
3,100000001,Silk Street,1,100000001,100000002,2355_0010,2017-09-29,"LINESTRING (-2.264742 53.48562800000001,-2.264...",168,None,48,None,None,None,None
4,100000001,Silk Street,1,100000001,100000002,0010_0025,2017-09-30,"LINESTRING (-2.264742 53.48562800000001,-2.264...",210,None,43,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4551,100000761,Leftbank,4,100000941,100000942,2325_2340,2017-09-29,"LINESTRING (-2.2532004 53.48182270000001,-2.25...",516,None,35,None,None,None,None
4552,100000761,Leftbank,4,100000941,100000942,2340_2355,2017-09-29,"LINESTRING (-2.2532004 53.48182270000001,-2.25...",612,None,36,None,None,None,None
4553,100000761,Leftbank,4,100000941,100000942,2355_0010,2017-09-29,"LINESTRING (-2.2532004 53.48182270000001,-2.25...",576,None,35,None,None,None,None
4554,100000761,Leftbank,4,100000941,100000942,0010_0025,2017-09-30,"LINESTRING (-2.2532004 53.48182270000001,-2.25...",588,None,40,None,None,None,None


In [17]:
measurement_utd.head(10)

,utd_link_id,corridor_id,corridor_link_order,from_node_id,to_node_id,time_period,date,geometry,volume,travel_time,speed,reference_speed,density,queue,notes
0,100000001,Silk Street,1,100000001,100000002,2310_2325,2017-09-29,"LINESTRING (-2.264742 53.48562800000001,-2.264...",240,None,40,None,None,None,None
1,100000001,Silk Street,1,100000001,100000002,2325_2340,2017-09-29,"LINESTRING (-2.264742 53.48562800000001,-2.264...",228,None,36,None,None,None,None
2,100000001,Silk Street,1,100000001,100000002,None,None,"LINESTRING (-2.264742 53.48562800000001,-2.264...",None,None,None,None,None,None,None
3,100000002,Silk Street,2,100000002,100000003,2310_2325,2017-09-29,"LINESTRING (-2.264694 53.486187,-2.264653 53.4...",240,None,40,None,None,None,None
4,100000002,Silk Street,2,100000002,100000003,2325_2340,2017-09-29,"LINESTRING (-2.264694 53.486187,-2.264653 53.4...",228,None,36,None,None,None,None
5,100000002,Silk Street,2,100000002,100000003,None,None,"LINESTRING (-2.264694 53.486187,-2.264653 53.4...",None,None,None,None,None,None,None
6,100000003,Silk Street,3,100000003,100000004,2310_2325,2017-09-29,"LINESTRING (-2.264653 53.48682,-2.264374 53.48...",240,None,40,None,None,None,None
7,100000003,Silk Street,3,100000003,100000004,2325_2340,2017-09-29,"LINESTRING (-2.264653 53.48682,-2.264374 53.48...",228,None,36,None,None,None,None
8,100000003,Silk Street,3,100000003,100000004,None,None,"LINESTRING (-2.264653 53.48682,-2.264374 53.48...",None,None,None,None,None,None,None
9,100000004,Silk Street,4,100000004,100000005,2310_2325,2017-09-29,"LINESTRING (-2.264374 53.48719000000001,-2.263...",240,None,40,None,None,None,None


In [27]:
print( __name__)
print(os.path.dirname(os.path.realpath('__file__')))
print(os.path.realpath('__file__'))
print(os.getcwd())

__main__
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS/__file__
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS
